In [14]:
import asyncio
from playwright.async_api import async_playwright

import pandas as pd
import re

In [15]:
async def get_text_content(h_elements):
  text_list = []
  for h_ele in h_elements:
    h_text = await h_ele.text_content()
    text_list.append(h_text.strip())
  return text_list

async def scrape_tweets(url: str):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context()
        page = await context.new_page()

        await page.goto(url, wait_until='load', timeout=60000)

        # Scroll untuk load lebih banyak tweets
        for _ in range(3):  # Ubah angka scroll tergantung mau load berapa banyak
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
            await asyncio.sleep(2)

        # Cari semua tweet, mereka ada di dalam <article> elements
        tweets_elements = await page.query_selector_all("article")

        # Disini kita mengambil elemen yang diinginkan, kalian boleh modifikasi untuk mengambil url tweet atau lainnya
        tweets = []
        for tweet_element in tweets_elements:
            tweet_userdate = await tweet_element.query_selector_all("a") #username, @userid, datetime
            tweet_userdate_texts = await get_text_content(tweet_userdate)

            tweet_likes = await tweet_element.query_selector_all('div[data-testid="like"]')

            tweet_textdiv = await tweet_element.query_selector('div[data-testid="tweetText"]')
            tweet_text = await get_text_content([tweet_textdiv])

            if tweet_text:
                tweets.append([tweet_userdate_texts, tweet_text])

        await browser.close()
        return tweets

async def getTweets(url):
    tweets = await scrape_tweets(url)
    for i, tweet in enumerate(tweets, 1):
        print(f"Tweet {i}:\n{tweet}\n{'='*80}")
    return tweets


# set the url to scrape from
url = "https://twitter.com/hacktiv8id?lang=en"

result = await getTweets(url)

Task exception was never retrieved
future: <Task finished name='Task-11' coro=<Connection.run() done, defined at C:\Users\whisn\AppData\Roaming\Python\Python39\site-packages\playwright\_impl\_connection.py:264> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\whisn\AppData\Roaming\Python\Python39\site-packages\playwright\_impl\_connection.py", line 271, in run
    await self._transport.connect()
  File "C:\Users\whisn\AppData\Roaming\Python\Python39\site-packages\playwright\_impl\_transport.py", line 135, in connect
    raise exc
  File "C:\Users\whisn\AppData\Roaming\Python\Python39\site-packages\playwright\_impl\_transport.py", line 123, in connect
    self._proc = await asyncio.create_subprocess_exec(
  File "c:\ProgramData\Anaconda3\lib\asyncio\subprocess.py", line 236, in create_subprocess_exec
    transport, protocol = await loop.subprocess_exec(
  File "c:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 1676, in subprocess_exec
    tra

NotImplementedError: 

In [ ]:
print(len(result)) # cek jumlah tweet

NameError: name 'result' is not defined

In [ ]:
# simpan hasil scrape ke csv
df_tweets = pd.DataFrame(columns=["User", "Handle", "Date", "TweetText"])

for one_tweet in result:
  [userdate, [text]] = one_tweet

  new_row = {"User":userdate[1], "Handle":userdate[2], "Date":userdate[3], "TweetText":text}

  df_tweets.loc[len(df_tweets)] = new_row

display(df_tweets)
df_tweets.to_csv("Twitter_hacktiv8_today.csv")

NameError: name 'result' is not defined